## Solving the 1-d Wave equation using physics-informed-neural-networks 

Tutorial from https://github.com/Mustafa-Kaddoura/NeuralPDE/blob/main/Tutorial_PINN_WaveEquation_GuidedWorkFlow_Finished.jl#L70

We will solve the 1d wave equation 
$$ 
\frac{\partial^2 u}{\partial t^2} = c^2 \frac{\partial^2 u}{\partial x^2}
$$ 
subject to the Dirichlet boundary conditions
$$
\begin{align*}
u(0,t) &= u(1,t) = 0 \\
u(x, t) &= x(1 - x) \quad \forall t > 0 \\
\frac{\partial u}{\partial t}(x, 0) &= 0 \quad \forall x \in [0, 1]
\end{align*}
$$

In [12]:
using NeuralPDE, Lux, Optimization, OptimizationOptimJL
using ModelingToolkit 
# import ModelingToolkit : Interval
# use ModelingToolkit.Interval instead

# STEP 1: DEFINE THE EQUATION (2D PDE)

In [6]:
# set up model parameters/variables -- uses ModelingToolkit
@parameters t x 
@variables u(..)
Dxx = Differential(x)^2
Dtt = Differential(t)^2 
Dt = Differential(t)

(::Differential) (generic function with 3 methods)

In [7]:
# define the equations and IC/BC
C = 1
eq = Dtt(u(t, x)) ~ C^2 * Dxx(u(t, x))
bcs = [u(t, 0) ~ 0.0, # for all t > 0 and x = 0
       u(t, 1) ~ 0.0, # for all t > 0 and x = 1
       u(0, x) ~ x*(1.0 - x), # for all x in [0, 1]
       Dt(u(0, x)) ~ 0.0] # for all x in [0, 1]

4-element Vector{Equation}:
 u(t, 0) ~ 0.0
 u(t, 1) ~ 0.0
 u(0, x) ~ x*(1.0 - x)
 Differential(t)(u(0, x)) ~ 0.0

In [9]:
# Time and Space and Domains
domains = [t ∈ ModelingToolkit.Interval(0.0, 1.0),
           x ∈ ModelingToolkit.Interval(0.0, 1.0)]

# Discretization
dx = 0.1

0.1

In [10]:
# Define a PDE system
indvar = [t,x]
depvar = u(t,x)
@named pde_system = PDESystem(eq, bcs, domains,indvar,depvar)

PDESystem
Equations: Equation[Differential(t)(Differential(t)(u(t, x))) ~ Differential(x)(Differential(x)(u(t, x)))]
Boundary Conditions: Equation[u(t, 0) ~ 0.0, u(t, 1) ~ 0.0, u(0, x) ~ x*(1.0 - x), Differential(t)(u(0, x)) ~ 0.0]
Domain: Symbolics.VarDomainPairing[Symbolics.VarDomainPairing(t, 0.0 .. 1.0), Symbolics.VarDomainPairing(x, 0.0 .. 1.0)]
Dependent Variables: u(t, x)
Independent Variables: Num[t, x]
Parameters: SciMLBase.NullParameters()
Default Parameter ValuesDict{Any, Any}()

------------------------------------------------
# STEP 2: CHOOSE a NEURAL NETWORK ARCHITECTURE

In [19]:

# Number of dimensions
dims = length(domains)

# Number of inner nodes
n = 16

# Multilayer-layer perceptron (3 layers, sigmoid activation function)
chain = Lux.Chain(Dense(dims, n, Lux.σ), Dense(n, n, Lux.σ), Dense(n, 1))
display(chain) 

# Discretizer
discretization = PhysicsInformedNN(chain, GridTraining(dx))
display(discretization)

# Convert the PDE system into an Optimization Problem using 'discretize'
prob = discretize(pde_system, discretization)

Chain(
    layer_1 = Dense(2 => 16, sigmoid_fast),  # 48 parameters
    layer_2 = Dense(16 => 16, sigmoid_fast),  # 272 parameters
    layer_3 = Dense(16 => 1),           # 17 parameters
)         # Total: 337 parameters,
          #        plus 0 states.

PhysicsInformedNN{GridTraining{Float64}, Nothing, NeuralPDE.Phi{Chain{NamedTuple{(:layer_1, :layer_2, :layer_3), Tuple{Dense{true, typeof(sigmoid_fast), typeof(glorot_uniform), typeof(zeros32)}, Dense{true, typeof(sigmoid_fast), typeof(glorot_uniform), typeof(zeros32)}, Dense{true, typeof(identity), typeof(glorot_uniform), typeof(zeros32)}}}, Nothing}, NamedTuple{(:layer_1, :layer_2, :layer_3), Tuple{NamedTuple{(), Tuple{}}, NamedTuple{(), Tuple{}}, NamedTuple{(), Tuple{}}}}}, typeof(NeuralPDE.numeric_derivative), Bool, Nothing, Nothing, Nothing, Base.Pairs{Symbol, Union{}, Tuple{}, NamedTuple{(), Tuple{}}}}(Chain(), GridTraining{Float64}(0.1), nothing, NeuralPDE.Phi{Chain{NamedTuple{(:layer_1, :layer_2, :layer_3), Tuple{Dense{true, typeof(sigmoid_fast), typeof(glorot_uniform), typeof(zeros32)}, Dense{true, typeof(sigmoid_fast), typeof(glorot_uniform), typeof(zeros32)}, Dense{true, typeof(identity), typeof(glorot_uniform), typeof(zeros32)}}}, Nothing}, NamedTuple{(:layer_1, :layer_2, :

OptimizationProblem. In-place: true
u0: ComponentVector{Float64}(layer_1 = (weight = [-0.3951249420642853 -0.35931530594825745; 0.38507020473480225 0.19643153250217438; … ; -0.441673219203949 -0.4893757700920105; -0.5357662439346313 -0.09209005534648895], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_2 = (weight = [-0.10293155163526535 0.08748529851436615 … -0.05611604079604149 -0.3545544445514679; 0.2209061086177826 0.26305022835731506 … -0.2563933730125427 -0.4019336998462677; … ; -0.3104294240474701 -0.3319527506828308 … -0.1949196308851242 0.22912701964378357; -0.35730013251304626 0.06772172451019287 … -0.29068589210510254 -0.014540646225214005], bias = [0.0; 0.0; … ; 0.0; 0.0;;]), layer_3 = (weight = [-0.38839301466941833 0.5047451257705688 … 0.2743741571903229 0.1821022778749466], bias = [0.0;;]))

In [22]:
#---
# Call back function
callback = function (p,l)
    #println("Current loss is: $l")
    return false
end


#13 (generic function with 1 method)

# STEP 4: SOLVE OPTIMIZATION PROBLEM

In [23]:
# Select an Optimizer: BFGS Algorithm
opt = OptimizationOptimJL.BFGS()

# Solve the Optimization Problem
res = Optimization.solve(prob, opt; callback = callback, maxiters=1200)
display(res)
phi = discretization.phi

u: ComponentVector{Float64}(layer_1 = (weight = [-1.838896524577054 -1.5441336882897496; 1.1963826520026686 1.8876113979955744; … ; -2.469418195070733 -2.4280343411627734; -1.6172291676087682 -1.0858700235305758], bias = [-1.2734725701979681; 2.4804323323558615; … ; -0.5135327758829323; -0.79796643035074;;]), layer_2 = (weight = [-1.1727387844044126 1.323354798921653 … 0.32018976359004536 -1.0301349812973914; 1.2526664628655948 -0.09568399368833186 … -1.4231335693680118 0.1476504604114746; … ; 0.328328651013151 0.06507974827672514 … 0.9960908673674814 0.637780047338786; 0.12548871474020307 0.023682664068447545 … -0.09584090920635482 0.35221174131299354], bias = [0.40030071383935956; 0.27484726067209253; … ; 1.0414073222304385; 0.34748721773653085;;]), layer_3 = (weight = [2.869336873314732 0.7868387973088713 … 0.5933328923606016 0.5621319187211921], bias = [-0.7418232464960857;;]))

NeuralPDE.Phi{Chain{NamedTuple{(:layer_1, :layer_2, :layer_3), Tuple{Dense{true, typeof(sigmoid_fast), typeof(glorot_uniform), typeof(zeros32)}, Dense{true, typeof(sigmoid_fast), typeof(glorot_uniform), typeof(zeros32)}, Dense{true, typeof(identity), typeof(glorot_uniform), typeof(zeros32)}}}, Nothing}, NamedTuple{(:layer_1, :layer_2, :layer_3), Tuple{NamedTuple{(), Tuple{}}, NamedTuple{(), Tuple{}}, NamedTuple{(), Tuple{}}}}}(Chain(), (layer_1 = NamedTuple(), layer_2 = NamedTuple(), layer_3 = NamedTuple()))

# STEP 5: PLOTS 

In [29]:
using Gnuplot
ts, xs = [infimum(d.domain):dx:supremum(d.domain) for d in domains]
function analytic_sol_func(t, x)
    sum([(8 / (k^3 * pi^3)) * sin(k * pi * x) * cos(C * k * pi * t) for k in 1:2:50000])
end

u_predict = reshape([first(phi([t, x], res.u)) for t in ts for x in xs],
                    (length(ts), length(xs)))
u_real = reshape([analytic_sol_func(t, x) for t in ts for x in xs],
                 (length(ts), length(xs)))
diff_u = abs.(u_predict .- u_real)

11×11 Matrix{Float64}:
 0.00232569   0.0021576    0.00144421   …  0.000343975  0.000276621
 0.00738056   0.00165589   0.00573464      0.00576845   0.0104885
 0.0058406    0.0038045    0.0034712       0.00752421   0.00874511
 0.000161547  0.000718332  0.00076676      0.00244872   0.0013943
 0.00471032   0.00287198   0.00130281      0.00351425   0.00569933
 0.00607078   0.00412726   0.00101275   …  0.00626007   0.00860393
 0.00334938   0.00219088   0.00101481      0.00423569   0.00593637
 0.00207916   0.00179137   0.000461211     0.00127142   0.000940861
 0.00710188   0.00473752   0.00348565      0.00634001   0.00816306
 0.00709255   0.00183433   0.00541525      0.00505471   0.00996455
 0.00379469   0.00305817   0.00113884   …  0.000414798  0.000506861

In [1]:
## Plot the results.

In [12]:
# dice problem  # random problem I found on the internet    
# A fair coin is tossed. If a head occurs, 1 die is rolled; if a tail occurs, 2 dice are rolled. Let Y be the total on the die or dice.
function expdice() 
    N = 100000
    _sum = 0 

    for i in 1:N 
        d = rand()  
        if d < 0.5 
            _sum += rand(1:6) 
        else 
            _sum += rand(1:6) + rand(1:6) 
        end 
    end
    return _sum / N 
end
expdice()


5.25328